In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pywoe
import pandas as pd
import sklearn.datasets
import numpy
import pickle

n_nas = 5
rng = numpy.random.default_rng(42)

iris = sklearn.datasets.load_iris()
X = pd.DataFrame(numpy.concatenate([iris.data, numpy.repeat(numpy.nan, 4 * n_nas).reshape(-1, iris.data.shape[1])], axis=0))
y = pd.Series(numpy.concatenate([(iris.target == 1).astype(int), numpy.array([rng.integers(0, 2) for _ in range(n_nas)])], axis=0))

categories = ['A', 'B', 'C', 'D', 'E', 'F']
probabilities = [
    [0.22, 0.2, 0.18, 0.16, 0.14, 0.1],
    [0.12, 0.16, 0.22, 0.22, 0.16, 0.12],
    [0.1, 0.14, 0.16, 0.18, 0.2, 0.22]
]

rng_cat = numpy.random.default_rng(42)
# X = numpy.concatenate([X, numpy.array([rng_cat.choice(categories, size=1, p=probabilities[target])[0] for target in y]).reshape(-1, 1)], axis=1)
X["categorical"] = numpy.array([rng_cat.choice(categories, size=1, p=probabilities[target])[0] for target in y])
# Add random NA to categorical
for i in range(10):
    X.loc[rng.integers(0, len(X)), "categorical"] = numpy.nan
X["static"] = 1

class IrisData:
    X = X
    y = y

iris_data = IrisData()

In [3]:

woe = pywoe.PyWOE(min_leave_freq=0.05, random_state=42)
woe.fit(iris_data.X, iris_data.y)
display(woe.splits())
woe.woe_summary()

/home/linards/Documents/pywoe/src/pywoe/pywoe.py:158: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cor = scipy.stats.spearmanr(x, y)[0]


{'0': {'cuts': [-inf, 4.9, 5.2, 5.5, inf],
  'na_label': '[5.5, inf) OR NA',
  'labels': ['[-inf, 4.9)', '[4.9, 5.2)', '[5.2, 5.5)', '[5.5, inf) OR NA']},
 '1': {'cuts': [-inf, inf],
  'na_label': '[-inf, inf) OR NA',
  'labels': ['[-inf, inf) OR NA']},
 '2': {'cuts': [-inf, 1.9, 3.3, inf],
  'na_label': '[1.9, 3.3) OR NA',
  'labels': ['[-inf, 1.9)', '[1.9, 3.3) OR NA', '[3.3, inf)']},
 '3': {'cuts': [-inf, 0.5, inf],
  'na_label': '[0.5, inf) OR NA',
  'labels': ['[-inf, 0.5)', '[0.5, inf) OR NA']},
 'categorical': {'cuts': ['A', 'B', 'C', 'D', 'E', 'F', 'NA'],
  'na_label': 'NA',
  'labels': ['A', 'B', 'C', 'D', 'E', 'F', 'NA']},
 'static': {'cuts': [-inf, inf], 'na_label': None, 'labels': ['[-inf, inf)']}}

,variable,freq,events,nas,non_events,eventrate,woe,iv,group
0,0,16,0.5,0.0,16.5,0.031250,-2.831531,0.418737,"[-inf, 4.9)"
1,0,25,4.5,0.0,21.5,0.180000,-0.898999,0.109164,"[4.9, 5.2)"
2,0,11,2.5,0.0,9.5,0.227273,-0.670025,0.029602,"[5.2, 5.5)"
3,0,103,46.5,5.0,57.5,0.451456,0.452644,0.141900,"[5.5, inf) OR NA"
4,1,155,52.5,5.0,103.5,0.338710,0.000000,0.000000,"[-inf, inf) OR NA"
5,2,48,0.5,0.0,48.5,0.010417,-3.905206,1.775967,"[-inf, 1.9)"
6,2,8,3.5,5.0,5.5,0.437500,0.217520,0.002782,"[1.9, 3.3) OR NA"
7,2,99,49.5,0.0,50.5,0.500000,0.649505,0.287068,"[3.3, inf)"
8,3,48,0.5,0.0,48.5,0.010417,-3.900612,1.782237,"[-inf, 0.5)"
9,3,107,52.5,5.0,55.5,0.490654,0.618529,0.282614,"[0.5, inf) OR NA"


In [4]:
woe.woe_summary().to_parquet("tests/reference_data/woe_summary_iris_p_005.parquet")
woe.transform(X).to_parquet("tests/reference_data/iris_fitted_p_005.parquet")
with open("tests/reference_data/splits_iris_p_005.pickle", "wb") as f:
    pickle.dump(woe.splits(), f)

In [5]:
X.head(10)

,0,1,2,3,categorical,static
0,5.1,3.5,1.4,0.2,E,1
1,4.9,3.0,1.4,0.2,C,1
2,4.7,3.2,1.3,0.2,E,1
3,4.6,3.1,1.5,0.2,D,1
4,5.0,3.6,1.4,0.2,A,1
5,5.4,3.9,1.7,0.4,F,1
6,4.6,3.4,1.4,0.3,E,1
7,5.0,3.4,1.5,0.2,E,1
8,4.4,2.9,1.4,0.2,A,1
9,4.9,3.1,1.5,0.1,C,1


In [6]:
# %%timeit
woe.transform(X).head(10)

,0,1,2,3,categorical,static
0,-0.898999,0.0,-3.905206,-3.900612,-0.007484,0.0
1,-2.831531,0.0,-3.905206,-3.900612,-0.021183,0.0
2,-2.831531,0.0,-3.905206,-3.900612,-0.007484,0.0
3,-2.831531,0.0,-3.905206,-3.900612,0.651762,0.0
4,-0.898999,0.0,-3.905206,-3.900612,-0.125943,0.0
5,-0.670025,0.0,-3.905206,-3.900612,-1.194065,0.0
6,-2.831531,0.0,-3.905206,-3.900612,-0.007484,0.0
7,-0.898999,0.0,-3.905206,-3.900612,-0.007484,0.0
8,-2.831531,0.0,-3.905206,-3.900612,-0.125943,0.0
9,-2.831531,0.0,-3.905206,-3.900612,-0.021183,0.0


In [7]:
woe_np = pywoe.PyWOE(min_leave_freq=0.05, random_state=42)
X_np = iris_data.X.drop(columns=["categorical", "static"])
woe_np.fit(numpy.array(X_np), y)
woe_np.woe_summary()

,variable,freq,events,nas,non_events,eventrate,woe,iv,group
0,0,16,0.5,0,16.5,0.031250,-2.831531,0.418737,"[-inf, 4.9)"
1,0,25,4.5,0,21.5,0.180000,-0.898999,0.109164,"[4.9, 5.2)"
2,0,11,2.5,0,9.5,0.227273,-0.670025,0.029602,"[5.2, 5.5)"
3,0,103,46.5,5,57.5,0.451456,0.452644,0.141900,"[5.5, inf) OR NA"
4,1,155,52.5,5,103.5,0.338710,0.000000,0.000000,"[-inf, inf) OR NA"
5,2,48,0.5,0,48.5,0.010417,-3.905206,1.775967,"[-inf, 1.9)"
6,2,8,3.5,5,5.5,0.437500,0.217520,0.002782,"[1.9, 3.3) OR NA"
7,2,99,49.5,0,50.5,0.500000,0.649505,0.287068,"[3.3, inf)"
8,3,48,0.5,0,48.5,0.010417,-3.900612,1.782237,"[-inf, 0.5)"
9,3,107,52.5,5,55.5,0.490654,0.618529,0.282614,"[0.5, inf) OR NA"


In [8]:
woe_np.transform(numpy.array(X_np)).to_parquet("tests/reference_data/iris_fitted_p_005_np.parquet")

In [9]:
woe_0025 = pywoe.PyWOE(min_leave_freq=0.025, random_state=42)
woe_0025.fit(iris_data.X, iris_data.y)
display(woe_0025.woe_summary())
display(woe_0025.splits())
woe_0025.transform(iris_data.X).to_parquet("tests/reference_data/iris_fitted_p_0025.parquet")
woe_0025.woe_summary().to_parquet("tests/reference_data/woe_summary_iris_p_0025.parquet")



/home/linards/Documents/pywoe/src/pywoe/pywoe.py:158: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cor = scipy.stats.spearmanr(x, y)[0]


,variable,freq,events,nas,non_events,eventrate,woe,iv,group
0,0,16,0.5,0.0,16.5,0.031250,-2.835997,0.417526,"[-inf, 4.9)"
1,0,25,4.5,0.0,21.5,0.180000,-0.903465,0.109520,"[4.9, 5.2)"
2,0,11,2.5,0.0,9.5,0.227273,-0.674491,0.029796,"[5.2, 5.5)"
3,0,5,2.5,5.0,3.5,0.500000,0.324038,0.004114,NA
4,0,98,44.5,0.0,54.5,0.454082,0.457799,0.137306,"[5.5, inf)"
5,1,150,50.5,0.0,100.5,0.336667,-0.014085,0.000190,"[-inf, inf)"
6,1,5,2.5,5.0,3.5,0.500000,0.337627,0.004563,NA
7,2,48,0.5,0.0,48.5,0.010417,-3.905206,1.775967,"[-inf, 1.9)"
8,2,8,3.5,5.0,5.5,0.437500,0.217520,0.002782,"[1.9, 3.3) OR NA"
9,2,99,49.5,0.0,50.5,0.500000,0.649505,0.287068,"[3.3, inf)"


{'0': {'cuts': [-inf, 4.9, 5.2, 5.5, inf],
  'na_label': 'NA',
  'labels': ['[-inf, 4.9)', '[4.9, 5.2)', '[5.2, 5.5)', 'NA', '[5.5, inf)']},
 '1': {'cuts': [-inf, inf], 'na_label': 'NA', 'labels': ['[-inf, inf)', 'NA']},
 '2': {'cuts': [-inf, 1.9, 3.3, inf],
  'na_label': '[1.9, 3.3) OR NA',
  'labels': ['[-inf, 1.9)', '[1.9, 3.3) OR NA', '[3.3, inf)']},
 '3': {'cuts': [-inf, 0.5, inf],
  'na_label': 'NA',
  'labels': ['[-inf, 0.5)', 'NA', '[0.5, inf)']},
 'categorical': {'cuts': ['A', 'B', 'C', 'D', 'E', 'F', 'NA'],
  'na_label': 'NA',
  'labels': ['A', 'B', 'C', 'D', 'E', 'F', 'NA']},
 'static': {'cuts': [-inf, inf], 'na_label': None, 'labels': ['[-inf, inf)']}}

In [10]:
woe_np_1 = pywoe.PyWOE(min_leave_freq=0.05, random_state=42)
woe_np_1.fit(numpy.array(X_np["0"]), y)
woe_np_1.woe_summary()
# woe_np_1.transform(numpy.array(X_np[:,1])).to_parquet("tests/reference_data/iris_fitted_p_005_np_1.parquet")
woe_np_1.woe_summary().to_parquet("tests/reference_data/woe_summary_iris_p_005_np_1.parquet")
woe_np_1.transform(numpy.array(X_np["0"])).to_parquet("tests/reference_data/iris_fitted_p_005_np_1.parquet")


In [11]:
numpy.array([1, 2, 3] * 10).reshape(-1, 1, 1).ndim

3

In [13]:
numpy.array([1, 2, 3] * 10).reshape(-1, 2).ndim

2

In [14]:
pywoe.PyWOE().fit(numpy.array([1, 2, 3] * 10), numpy.array([1, 2, 3] * 10).reshape(-1, 1, 1))

ValueError: y must be a 1D array, got 3D array.